In [113]:
import os,sys
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/13.TrafficPrediction')
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pyplot as plt # to visualize data
from pandas.tools.plotting import autocorrelation_plot # to visualize and configure the parameters of ARIMA model
from statsmodels.tsa.arima_model import ARIMA # to make an ARIMA model that fits the data

df_train=pd.read_csv('train_aWnotuB.csv',parse_dates=['DateTime'])
df_test=pd.read_csv('test_BdBKkAj.csv',parse_dates=['DateTime'])

monthly_regression_params={1: [2.93, 18.59], 2: [0.9, 6.18], 3: [0.5, 8.65], 4: [0.17, 5.19]}

In [130]:
def get_vars(df1):
    df=df1.copy()
    df['year']=df['DateTime'].dt.year
    df['month']=df['DateTime'].dt.month
    df['day']=df['DateTime'].dt.day
    df['hour']=df['DateTime'].dt.hour
    df['day_of_week']=df['DateTime'].dt.dayofweek
    df['week_nbr']=df['DateTime'].dt.week
    df['flag_sunday']=(df['day_of_week']==6).astype(np.int64)
    df['flag_saturday']=(df['day_of_week']==5).astype(np.int64)
    df['flag_friday']=(df['day_of_week']==4).astype(np.int64)
    df['flag_monday']=(df['day_of_week']==0).astype(np.int64)
    df['flag_weekday']=(df['day_of_week']<=4).astype(np.int64)
    df['flag_junction_1']=(df['Junction']==1).astype(np.int64)
    df['flag_junction_2']=(df['Junction']==2).astype(np.int64)
    df['flag_junction_3']=(df['Junction']==3).astype(np.int64)
    df['flag_junction_4']=(df['Junction']==4).astype(np.int64)
    df['flag_last_day_of_month']=(df['day']==df['month'].map({1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31})).astype(np.int64)
    df['encoded_month']=12*df['year']+df['month']-24191
    df['date']=10000*df['year']+100*df['month']+df['day']
    df['encoded_date']=(df['DateTime']-date(2015,11,1)).dt.days
    slope=df['Junction'].map(lambda x: monthly_regression_params[x][0])
    intercept=df['Junction'].map(lambda x: monthly_regression_params[x][1])
    df['monthly_avg']=np.round(intercept+slope*df['encoded_month'])
    return df

train=get_vars(df_train)
train_oct=train[train['month']==10]
train=pd.concat([train,train_oct,train_oct])
train=train.sort_values(by='DateTime')


In [131]:
params = {1:[7,1,1],2:[7,1,1],3:[4,1,0],4:[7,1,1]}

arima_prediction=[]
for key, group in train.groupby('Junction'):
    data = np.array(group.groupby('encoded_date')['Vehicles'].mean())
    result = None
    arima = ARIMA(data,params[key])
    result = arima.fit(disp=False)
    #print(result.params)
    pred = result.predict(2,len(data)-1,typ='levels')
    train_prediction=pd.Series([0,0]+list(pred),index=group['encoded_date'].drop_duplicates().sort_values())
    test_prediction=pd.Series(result.predict(len(data),len(data)+122,typ='levels'),index=map(int,range(608,731)))
    arima_prediction.append(train_prediction.append(test_prediction))

arima_daily_predictions=pd.DataFrame(arima_prediction,index=[1,2,3,4]).T.stack().to_dict()


C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [132]:
ins=train.head(int(0.95*len(train)))
oos=train.tail(len(train)-len(ins))
oot=get_vars(df_test)
ins['Vehicles_clean']=np.clip(ins['Vehicles'],np.percentile(ins['Vehicles'],0.01),np.percentile(ins['Vehicles'],0.99))
hourly_avg=ins.groupby(['hour','Junction'])['Vehicles_clean'].mean().unstack()
hourly_avg=(hourly_avg/hourly_avg.max()).stack()
hourly_avg=hourly_avg.map(lambda x: 'night' if x<0.70 else 'peak' if x>0.9 else 'regular').to_dict()

get_avgs=lambda x: ins.groupby(x)['Vehicles'].mean()

def get_other_vars(df):
    df['time_tag']=df.apply(lambda x: hourly_avg[x['hour'],x['Junction']],axis=1)
    df['flag_peak']=(df['time_tag']=='peak').astype(np.int64)
    df['flag_night']=(df['time_tag']=='night').astype(np.int64)
    df['flag_regular']=(df['time_tag']=='regular').astype(np.int64)
    for var,level in [['month_timetag',['month','time_tag']],['month_hour',['month','hour']],\
                     ['week_timetag',['week_nbr','time_tag']],['week_hour',['week_nbr','hour']],\
                     ['weekday_timetag',['day_of_week','time_tag']],['weekday_hour',['day_of_week','hour']],\
                     ['weekend_timetag',['flag_weekday','time_tag']],['weekend_hour',['flag_weekday','hour']],\
                     ['week',['week_nbr']],['month',['month']],['weekday',['day_of_week']]]:
        avgs=get_avgs(level).reset_index().rename(columns={'Vehicles':'avg_%s'%var})
        df=pd.merge(df,avgs,on=level,how='left')
    df['arima_prediction']=np.round(df.apply(lambda x: arima_daily_predictions[x['encoded_date'],x['Junction']],axis=1))
    return df
ins=get_other_vars(ins)
oos=get_other_vars(oos)
oot=get_other_vars(oot)


C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [133]:
print ins.columns
model_cols=[u'Junction',u'year', u'month', u'day',u'hour', u'day_of_week', u'week_nbr']+\
            [x for x in ins.columns if 'flag' in x]+\
            [x for x in ins.columns if 'avg' in x]

model_cols=['monthly_avg','avg_weekday_hour','week_nbr','avg_week_hour','day','avg_week_timetag','day_of_week','Junction','hour','year']

T_train_xgb = xgb.DMatrix(ins[model_cols], ins['Vehicles'])

xgtrain = xgb.DMatrix(ins[model_cols], ins['Vehicles'])
xgvalid = xgb.DMatrix(oos[model_cols], oos['Vehicles'])
xgtest = xgb.DMatrix(oot[model_cols])
watchlist  = [(xgvalid,'eval'), (xgtrain,'train')]

param = {'max_depth':3, 'eta':0.01, 'silent':1, 'objective':'reg:linear', 'min_child_weight':4,\
         'nthread':3, 'gamma': 0.0,'subsample':0.75, 'colsample_bytree':0.9, 'learning_rate':0.1,'reg_alpha':0.1}
print param
param['eval_metric'] = 'rmse'
plst = param.items()
evallist = [(xgvalid,'oos'), (xgtrain,'ins')]
model = xgb.train(plst, xgtrain, 200, evallist)
ins_pred_actual  = model.predict(xgtrain)
oos_pred_actual = model.predict(xgvalid)
oot_pred_actual = model.predict(xgtest)


Index([u'DateTime', u'Junction', u'Vehicles', u'ID', u'year', u'month', u'day',
       u'hour', u'day_of_week', u'week_nbr', u'flag_sunday', u'flag_saturday',
       u'flag_friday', u'flag_monday', u'flag_weekday', u'flag_junction_1',
       u'flag_junction_2', u'flag_junction_3', u'flag_junction_4',
       u'flag_last_day_of_month', u'encoded_month', u'date', u'encoded_date',
       u'monthly_avg', u'Vehicles_clean', u'time_tag', u'flag_peak',
       u'flag_night', u'flag_regular', u'avg_month_timetag', u'avg_month_hour',
       u'avg_week_timetag', u'avg_week_hour', u'avg_weekday_timetag',
       u'avg_weekday_hour', u'avg_weekend_timetag', u'avg_weekend_hour',
       u'avg_week', u'avg_month', u'avg_weekday', u'arima_prediction'],
      dtype='object')
{'reg_alpha': 0.1, 'learning_rate': 0.1, 'colsample_bytree': 0.9, 'silent': 1, 'nthread': 3, 'min_child_weight': 4, 'subsample': 0.75, 'eta': 0.01, 'objective': 'reg:linear', 'max_depth': 3, 'gamma': 0.0}
[0]	oos-rmse:39.4032	ins-rmse

[185]	oos-rmse:8.48387	ins-rmse:5.25769
[186]	oos-rmse:8.50383	ins-rmse:5.25598
[187]	oos-rmse:8.49786	ins-rmse:5.25471
[188]	oos-rmse:8.49683	ins-rmse:5.25241
[189]	oos-rmse:8.49841	ins-rmse:5.25174
[190]	oos-rmse:8.49703	ins-rmse:5.24993
[191]	oos-rmse:8.49897	ins-rmse:5.24952
[192]	oos-rmse:8.52068	ins-rmse:5.24646
[193]	oos-rmse:8.52814	ins-rmse:5.24459
[194]	oos-rmse:8.51796	ins-rmse:5.24309
[195]	oos-rmse:8.52095	ins-rmse:5.24255
[196]	oos-rmse:8.53123	ins-rmse:5.23718
[197]	oos-rmse:8.52782	ins-rmse:5.23581
[198]	oos-rmse:8.52915	ins-rmse:5.23559
[199]	oos-rmse:8.53444	ins-rmse:5.2343


In [63]:
param_test1 = {
 'reg_alpha':[0.05, 0.1, 0.5]
          }
gsearch1 = GridSearchCV(estimator = xgb.sklearn.XGBRegressor( learning_rate =0.01, n_estimators=100, max_depth=3,
 min_child_weight=4, gamma=0, subsample=0.75, colsample_bytree=0.9,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, n_jobs=1,iid=False, cv=5)
gsearch1.fit(ins[model_cols], ins['Vehicles'])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.83365, std: 0.06232, params: {'reg_alpha': 0.05},
  mean: 0.83379, std: 0.06242, params: {'reg_alpha': 0.1},
  mean: 0.83086, std: 0.06398, params: {'reg_alpha': 0.5}],
 {'reg_alpha': 0.1},
 0.8337894402698558)

In [134]:
oot['Vehicles']=np.round(model.predict(xgb.DMatrix(oot[model_cols])))
oot[['ID','Vehicles']].to_csv('XGBoost1.csv',index=False)

ins['score']=np.round(model.predict(xgb.DMatrix(ins[model_cols])))
ins['error']=np.abs(ins['score']-ins['Vehicles'])
ins['error_rank']=np.round(ins['error'].rank(pct=True),3)
ins.to_csv('chk1.csv')


IOError: [Errno 13] Permission denied: 'chk1.csv'

In [117]:
imp_days=ins.groupby(['Junction','encoded_date','month','day'])['Vehicles'].mean().reset_index()
imp_days_prev=imp_days.drop(['month','day'],axis=1)
imp_days_prev['encoded_date']=imp_days['encoded_date']+1
imp_days=pd.merge(imp_days,imp_days_prev,on=['Junction','encoded_date'])
imp_days['Delta']=100*np.abs(imp_days['Vehicles_x']/imp_days['Vehicles_y']-1)
print imp_days['Delta'].describe(percentiles=[0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99])
imp_days=imp_days[imp_days['Delta']>25]
imp_days=imp_days.groupby(['month','day'])['Delta'].count()
imp_days=imp_days[imp_days>3].reset_index()
imp_days['flag_imp_day']=1

#ins=pd.merge(ins,imp_days,on=['month','day'])

count    1901.000000
mean       17.025470
std        18.220164
min         0.000000
1%          0.130804
5%          0.865801
10%         1.633987
25%         4.195804
50%        10.360360
75%        24.468085
90%        42.657343
95%        55.084746
99%        77.840909
max       170.833333
Name: Delta, dtype: float64


In [135]:
ins.groupby(['month'])['score'].mean().fillna(0)

month
1     19.756720
2     22.856575
3     23.303379
4     24.274206
5     24.673388
6     19.885893
7     21.552420
8     22.075716
9     25.739815
10    26.667562
11    19.453703
12    18.938620
Name: score, dtype: float32